## Import

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from catboost import CatBoostClassifier

import seaborn as sns
import matplotlib.pyplot as plt

## Function

In [2]:
def get_cat_barplot(df, cat_feature):
  _data = (df[cat_feature].value_counts().reset_index())
  sns.barplot(_data, x=cat_feature, y='count')
  plt.xticks(rotation=90)
  plt.show()

def get_box_hist_plot(df, cat_feature, target):
  _data = (df[cat_feature].value_counts().reset_index())
  sns.boxplot(df, x=cat_feature, hue=target)
  plt.xticks(rotation=90)
  plt.show()
  sns.histplot(df, x=cat_feature, hue=target)
  plt.xticks(rotation=90)
  plt.show()
  return df[cat_feature].describe()

## Data loading

In [ ]:
data_train = pd.read_csv(r'train_events.csv')
data_train.head(5)

In [ ]:
data_target = pd.read_csv(r'train_targets.csv')
data_target.head(5)

In [ ]:
data_video = pd.read_csv(r'video_info_v2.csv')
data_video.head(5)

## Merging data

In [6]:
merged_data = pd.merge(data_train, data_video, on='rutube_video_id', how='outer')
all_train_data = pd.merge(merged_data, data_target, on='viewer_uid', how='outer')

In [ ]:
get_box_hist_plot(all_train_data, 'total_watchtime', 'age_class')

In [ ]:
get_box_hist_plot(all_train_data, 'total_watchtime', 'sex')

In [ ]:
get_box_hist_plot(all_train_data, 'category', 'sex')

По графику видно, что женщин на площадке больше и они в основном смотрят телепередачи и сериалы

In [ ]:
get_box_hist_plot(all_train_data, 'category', 'age_class')

По данному графику можно увидеть, какой контент предпочитают смотреть данные возрастные категории 
Например, людям от 9 до 20 лет больше нравится смотреть сериалы и телепередачи, а пользователям от 40 до 60 лет - наука, авто-мото и эзотерика

## Data train EDA

In [ ]:
data_train.shape

In [ ]:
data_train.info()

In [ ]:
get_cat_barplot(data_train, 'region')

In [ ]:
data_train['region'].value_counts().to_dict()

    Среди регионов преобладает Москва
    Самые малочисленные по данным регионы Астрахань, Архангельская область, Ставропольский край
    Возможно имеет смысл часть рекламы рутюб нацеливать на регионы, с наименьшим использованием (больше людей узнают - больше людей смотрят)

In [ ]:
get_cat_barplot(data_train, 'ua_device_type')

С телефонов рутюба просматривают чаще

In [ ]:
get_cat_barplot(data_train, 'ua_client_type')

Логично, что исходя из графика выше из мобильного приложения смотрят чаще

In [ ]:
get_cat_barplot(data_train, 'ua_os')

Так как на мак и ios ютуб нельзя скачать через appstore, логично, что преобладают android и windows

In [ ]:
get_cat_barplot(data_train, 'ua_client_name')

Большая часть просмотров - через сам рутуб

Время в секундах, приведем к часам

In [ ]:
data_train['total_watchtime'] = data_train['total_watchtime'] / 3600

In [ ]:
get_box_hist_plot(data_train, 'total_watchtime', None)

Видим ОГРОМНЫЕ выбросы, в дальнейшем будем работать над их устранением

In [ ]:
data_train['total_watchtime'].min()

In [ ]:
data_train['total_watchtime'].max()

In [ ]:
data_train['total_watchtime'].mean()

In [ ]:
data_train['total_watchtime'].median()

## Data video EDA

In [ ]:
data_video.shape

In [ ]:
data_video.info()

Длительность указана в милисекундах, переведем в минуты

In [ ]:
get_box_hist_plot(data_video, 'duration', None)

In [ ]:
data_video['duration'].min()

In [ ]:
data_video['duration'].max()

In [ ]:
data_video['duration'].mean()

In [ ]:
data_video['duration'].median()

In [ ]:
get_cat_barplot(data_video, 'category')

## Data target EDA

In [ ]:
data_target.shape

In [ ]:
data_target.columns

In [ ]:
data_target.info()

In [ ]:
get_cat_barplot(data_target, 'age_class')

In [ ]:
get_cat_barplot(data_target, 'sex')

In [ ]:
get_box_hist_plot(data_target, 'age', None)

In [ ]:
data_target['age'].mode()

Самый частый возраст пользователя - 24 года

## Mergind data EDA

In [ ]:
all_train_data.shape

In [ ]:
all_train_data.columns

In [ ]:
all_train_data.describe()

In [ ]:
all_train_data.info()

## Проверим на пропуски и дубли

In [ ]:
all_train_data.isna().sum()

Пропуски в таргете, поэтому делаем drop данных значений, так как записи без таргета для обучения смысла не имеют

In [ ]:
all_train_data = all_train_data.dropna()

In [ ]:
all_train_data.isna().sum()

In [ ]:
all_train_data.duplicated().sum()

Полных дубликатов - нет

In [ ]:
all_train_data['sex'] = all_train_data['sex'].replace({'female': 0, 'male': 1})
all_train_data['sex'].value_counts()

Привели один из таргетов к бинарной классификации (0 - женщина, 1 - мужчина)

## Corr

In [ ]:
merged_data = merged_data.drop(columns='title')

In [ ]:
pearson_corr = merged_data.corr(method='pearson')

sns.set(style="white")

plt.figure(figsize=(20, 16))

sns.heatmap(pearson_corr, annot=True, cmap='coolwarm', linewidths=.5, annot_kws={"size": 10})

plt.title('Коэффициент корреляции Пирсона', fontsize=20)
plt.show()

In [ ]:
pearson_corr = merged_data.corr(method='spearman')

sns.set(style="white")

plt.figure(figsize=(20, 16))

sns.heatmap(pearson_corr, annot=True, cmap='coolwarm', linewidths=.5, annot_kws={"size": 10})

plt.title('Коэффициент корреляции Спирмана', fontsize=20)
plt.show()

In [ ]:
pearson_corr = merged_data.corr(method='kendall')

sns.set(style="white")

plt.figure(figsize=(20, 16))

sns.heatmap(pearson_corr, annot=True, cmap='coolwarm', linewidths=.5, annot_kws={"size": 10})

plt.title('Коэффициент корреляции Кендалла', fontsize=20)
plt.show()